# Uniformar los datos satelitales de NASA - diarios por estación

In [1]:
import pandas as pd
import numpy as np
import os

### Funciones

In [2]:
'''Definir perido de los datos'''
fecha_ini = '1981-01-01'
fecha_fin = '2020-12-31'

'''devolver la fecha en base al año y el numero de dia'''
def aniodia(anio, dia=1):
    fecha = pd.to_datetime(dia-1, unit='D', origin=str(anio))
    return fecha.date()

'''devolver el rango de fecha en base al año, dia inicial y periodo'''
def rango_fecha1(anio, dia, periodo):
    rango = pd.date_range(start = aniodia(anio, dia), freq = 'D', periods = periodo)
    return rango

'''leer los archivos en un carpeta dada SI son archivos CSV y si su nombre comienza con NASA'''
def leer_dir(path):
    with os.scandir(path) as ficheros:
        ficheros = [fichero.name for fichero in ficheros if fichero.is_file() \
                    and fichero.name.endswith('.csv') and fichero.name.startswith('nasa')]
    ficheros.sort()
    return ficheros

'''crear un DF con una sola columna en base a otro DF y una columna del mismo'''
def creardf(data, col):
    dat = pd.DataFrame()
    dat[col] = data[col]
    return dat

'''guardar un DF en base a un nombre y un directorio'''
def guardar(data, nom, path='../datos/1.0.variables/'):
    pth = path + nom
    data.to_csv(pth, index = False)

### Generar archivos con datos uniformados 

In [3]:
dir_f4 = '../datos/1.3.formato5_nasa/'
archivos = leer_dir(dir_f4)

In [4]:

for p in archivos:
    ruta = dir_f4 + p
    data = pd.read_csv(ruta, skiprows=13)
    
    '''leer anio y dia de inicio'''
    anio = data.iloc[0,0]
    dia = data.iloc[0,1]
    dias = data.shape[0]
    
    '''adicionar la columna fecha'''
    data['fecha'] = rango_fecha1(anio, dia, dias)
    
    '''cambiar de nombre las columnas'''
    data = data.rename(columns={"fecha": "fecha", "PRECTOTCORR": "pp", "T2M_MAX":\
                                "tmax", "T2M_MIN": "tmin", "T2M": "tmed", "RH2M": "hmed"})

    '''reordenar las columnas y eliminar columnas YEAR y DOY'''
    data = data.reindex(['fecha','pp','tmax','tmin','tmed','hmed'], axis=1)    
    
    '''periodo de datos'''
    data = data[(data['fecha'] >= fecha_ini) & (data['fecha'] <= fecha_fin)]
    
    '''guardar archivo'''
    dir_guardar = '../datos/1.0.formato_csv_corregido/' + p
    data.to_csv(dir_guardar, index = False)
    
    

### Generar archivos para cada variable meteorológica 

In [5]:
'''llamar a la funcion LEER_DIR() para recuperar archivos de un directorio'''
lista_arch = leer_dir('../datos/1.0.formato_csv_corregido/')

'''leer un DF en forma temporal'''
pin = '../datos/1.0.formato_csv_corregido/' + lista_arch[0]
temp = pd.read_csv(pin)

'''crear DF para cada variable, tomando como base el DF TEMP'''
dfpp = creardf(temp,['fecha'])
dftmx = creardf(temp,['fecha'])
dftmn = creardf(temp,['fecha'])
dftmd = creardf(temp,['fecha'])
dfhmd = creardf(temp,['fecha'])

'''crear una lista con los nombres de las columnas'''
nom_col = ['fecha']

for p in lista_arch:
    pin = '../datos/1.0.formato_csv_corregido/' + p
    file = pd.read_csv(pin)
    dfpp = pd.concat([dfpp, file['pp']], axis=1)
    dftmx = pd.concat([dftmx, file['tmax']], axis=1)
    dftmn = pd.concat([dftmn, file['tmin']], axis=1)
    dftmd = pd.concat([dftmd, file['tmed']], axis=1)
    dfhmd = pd.concat([dfhmd, file['hmed']], axis=1)
    
    cad = p.split('_')
    nom_col.append(str(cad[1]))
    
dfpp.columns = nom_col
dftmx.columns = nom_col
dftmn.columns = nom_col
dftmd.columns = nom_col
dfhmd.columns = nom_col

guardar(dfpp, 'nasa_diario_pp.csv')
guardar(dftmx, 'nasa_diario_tmax.csv')
guardar(dftmn, 'nasa_diario_tmin.csv')
guardar(dftmd, 'nasa_diario_tmed.csv')
guardar(dfhmd, 'nasa_diario_hmed.csv')

In [6]:
'''Dataframe precipitacion con datos de NASA'''
print(dfpp.head(5))

        fecha    100    101    102    105    106    107    108    111    112  \
0  1981-01-02   0.77   0.35   0.41   0.77   0.58   0.36   0.55   0.22   0.58   
1  1981-01-03   3.17   1.88   2.50   3.17   2.90   2.53   2.67   1.32   2.90   
2  1981-01-04  17.93  16.86  19.41  17.93  16.50  18.94  15.23  10.95  16.50   
3  1981-01-05  11.58   8.82  10.20  11.58   7.53   9.89   6.97   7.15   7.53   
4  1981-01-06   1.61   3.41   1.71   1.61   2.61   1.77   2.48   1.60   2.61   

   ...    118    119    120    121    200    201   202    203   204    205  
0  ...   0.23   0.35   0.30   0.58   3.05   3.31  0.92   1.72  1.61   1.69  
1  ...   2.02   1.88   2.01   2.90   7.27   8.05  2.17   4.86  3.82   4.51  
2  ...  15.43  16.86  19.44  16.50  10.77  11.30  3.28  12.88  4.54  17.88  
3  ...   7.43   8.82   7.54   7.53   4.29   4.13  1.34   5.48  2.01   8.67  
4  ...   1.18   3.41   4.15   2.61  11.09  12.26  3.38   6.59  5.48   4.35  

[5 rows x 25 columns]
